<h1 align=center><font size = 5> Assignment week 3, part 2: Neighborhoods in Toronto City </font></h1>

In [2]:
!apt-get install python3-bs4
!pip install lxml

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


In [3]:
!easy_install html5lib

Searching for html5lib
Best match: html5lib 0.9999999
Adding html5lib 0.9999999 to easy-install.pth file

Using /home/jupyterlab/conda/envs/python/lib/python3.6/site-packages
Processing dependencies for html5lib
Finished processing dependencies for html5lib


In [4]:
import numpy as np
import pandas as pd 
import requests
data_link = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [5]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(data_link,'lxml')
#print(soup.prettify()) #Print index page of the link

toronto_table = soup.find('table', {'class':'wikitable sortable'})
#toronto_table #I can print our requested table of the project
data_table = toronto_table.find_all('tr')

In [6]:
#Get raw data
table = []
for tr in data_table:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    table.append(row)
df=pd.DataFrame(table, columns=["PostalCode", "Borough", "Neighborhood"])
df.head(5)
#You can see raw data below, so we need too clean it

,PostalCode,Borough,Neighborhood
0,None,None,None
1,M1A,Not assigned,Not assigned\n
2,M2A,Not assigned,Not assigned\n
3,M3A,North York,Parkwoods\n
4,M4A,North York,Victoria Village\n


## Data processing

In [7]:
#Drop 'Not assigned' cells from Borough
df=df[df.Borough!='Not assigned'] 
df=df.drop(df.index[0])
df_new=df.reset_index(drop=True)
df_new['Neighborhood']=df_new.Neighborhood.str.replace('\n','')

#Group dublicated PostalCodes and get the sum of Neighborhood  
df_new=df_new.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(", ".join).reset_index() 
#Assign the Borough value where Neighborhood is 'Not assigned'
df_new.Neighborhood = np.where(df_new.Neighborhood=='Not assigned', df_new.Borough, df_new.Neighborhood)

df_new.head(100)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
95,M9C,Etobicoke,"Bloordale Gardens, Eringate, Markland Wood, Ol..."
96,M9L,North York,Humber Summit
97,M9M,North York,"Emery, Humberlea"
98,M9N,York,Weston


In [8]:
df_new.shape

(103, 3)

# Summarize of part 1 workflow
1. I used numpy, pandas and request packages in above part. I also use BeautifulSoup to fetch the link and then I created pandas Data Frame from Wikipedia which consist of three columns: PostalCode, Borough, and Neighborhood
2. In first step of data-preprocessing I dropped 'Not assigned' part from Borough then I grouped dublicated variables of PostalCode and get sum of the values with seperated comma in Neighborhood column.
3. I assigned Borough value where the Neighborhood is 'Not assigned'
4. In last, you can see head of my data and the shape of it (103,3)

# Part 2: Latitude & Longitude Informations of Postal Codes

In [13]:
#Get Latitude & Longitude Informations of Postal Codes from CSV file
postalCode_url = "http://cocl.us/Geospatial_data"
geo_info = pd.read_csv(postalCode_url)
geo_info.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
# Add latitude and longitude into the data frame
df_new['Latitude'] = np.where(geo_info['Postal Code'] == df_new.PostalCode, geo_info.Latitude, "NA").astype(float)
df_new['Longitude'] = np.where(geo_info['Postal Code'] == df_new.PostalCode, geo_info.Longitude, "NA").astype(float)
df_new.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [15]:
Toronto = df_new.Borough.str.contains("Toronto")
df_new[Toronto].head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
